In [1]:
import run_api

Obtained my token successfully!
{'data': 2, 'error': 'task_on_executing', 'error_Description': "Task is running! You don't need to start again."}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 2, 'error': 'task_on_executing', 'error_Description': "Task is running! You don't need to start again."}
NFiles Not exported, Could be empty or a problem with octoparse. Considered empty for now
secondextract Not exported, Could be empty or a problem with octoparse. Considered empty for now
thirdextract Not exported, Could be empty or a problem with octoparse. Considered empty for now


In [2]:
import utils
from ner import NER

csv = utils.csv
pd = utils.pd
np = utils.np
nltk = utils.nltk
itertools = utils.itertools
re = utils.re

[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
xls = pd.ExcelFile('strings stems.xlsx')
first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
tutoring = pd.read_excel(xls, 'Tutoring').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')
at_least_another= pd.read_excel(xls, 'at least another').dropna(axis=1, how='all').dropna(axis=0, how='all')

payments = pd.read_excel(xls, 'payments').dropna(axis=1, how='all').dropna(axis=0, how='all')
contract = pd.read_excel(xls, 'contract').dropna(axis=1, how='all').dropna(axis=0, how='all')


utils.pre_process_df_keywords(parent_second)
utils.pre_process_df_keywords(first)
utils.pre_process_df_keywords(bad_keywords)



In [4]:
extractor = NER()

In [5]:
Nfiles_df_list, first_extract, second_extract, third_extract = run_api.Nfiles_df_list, run_api.first_extract_df, run_api.second_extract_df, run_api.third_extract_df
first_extract.rename(inplace=True, columns={"field1_Text_Text": "title", "field1_Link_Link": "link"})
first_extract = run_api.cleanup_df(first_extract,"title")
second_extract = run_api.cleanup_df(second_extract,"field1")
third_extract = run_api.cleanup_df(third_extract,"email")

extracted_pay = utils.extract_pay_info(second_extract,extractor)

In [6]:
second_extract_pay_appended = second_extract.reset_index().merge(extracted_pay.reset_index(), on="index",how="left").drop(["index","field","Res"],axis=1)
all_info = second_extract_pay_appended.merge(third_extract, on= "field1", how="left").dropna().drop_duplicates(subset = "email")
all_info.rename(columns={"field1": "title", "field2":"body", "field3": "compensation","email": "email", "Decesion":"pay_amount"}, inplace=True)

In [95]:
pay_amount, rate, cl_email = -1, "NA", ""

title = "Graphic Designer Wanted For Weekly Flyer And Special Flyers"

body = """QR Code Link to This Post
            
        
We are doing a weekly Techno party and we're looking for Graphic designers who do great content in that genre. We are also looking for graphic designers who have international appeal and can make an array of different types of designs. Pay for our weekly parties is a minimum of $30 per event (but may vary depending on the quality of work per designer). For special jpegs we are negotiable. For consideration please submit your name, contact number,  design work, age,  and any other reference of work.

"""     
raw_job_post = title + "\n\n" + body
generated_email = ""

raw_job_post = raw_job_post.replace("QR Code Link to This Post", "").lower()

job_post = utils.pre_process(raw_job_post)

if len(raw_job_post) < 5 and len(raw_job_post.split()) <2:
    print(f"Breaking!! because is too short\n\n")
#     continue


if any(utils.step_1(raw_job_post, utils.get_two_word(job_post), bad_keywords)):
    print(f"Breaking!! because {utils.step_1(raw_job_post, utils.get_two_word(job_post), bad_keywords)} detected\n\n") 
#     continue

first_result = utils.find_workers(raw_job_post, job_post, first, at_least_another)[0]
first_result

array(['a graphic designer'], dtype=object)

In [40]:
highest_match = utils.find_workers(raw_job_post, job_post, first, at_least_another)[1]
parent_second_result = utils.find_workers(raw_job_post, job_post, parent_second)[0]
tutoring_result = utils.find_workers(raw_job_post, job_post, tutoring)[0]
parent_second_result

array(['photoshop', 'wordpress', 'indesign', 'russian'], dtype=object)

In [29]:
pay_method = utils.find_matching_key_word(job_post,payments)
contract_type = utils.find_matching_key_word(job_post,contract)


formatted_first = ""
formatted_second_parent = ""
formatted_pay = ""
formatted_tutoring = ""
formatted_pay_method = ""
formatted_contract_type = ""
formatted_sample = ""

is_sample = False
is_tutor = any(tutoring_result)
is_payment_method = any(pay_method)
is_contract_type = any(contract_type)

if any(first_result):

    if "someone" not in first_result:
        if len(first_result) == 1: formatted_first = first_result[0]
        elif len(first_result) == 2: formatted_first = f"{first_result[0]} and {first_result[1]}"
        elif len(first_result) == 3: formatted_first = f"{first_result[0]}, {first_result[1]} and {first_result[2]}"
        else: formatted_first = "someone"
    else: formatted_first = "someone"


    if any(parent_second_result):
        if len(parent_second_result) == 1: formatted_second_parent = parent_second_result[0]
        elif len(parent_second_result) == 2: formatted_second_parent = f"{parent_second_result[0]} and {parent_second_result[1]}"
        elif len(parent_second_result) == 3: formatted_second_parent = f"{parent_second_result[0]}, {parent_second_result[1]} and {parent_second_result[2]}"
        else: formatted_second_parent = "many things"
        is_sample = any(['karim' in parent_second[i].values for i in parent_second_result])


else:
    print(f"Breaking because no first parent found")
    #continue



formatted_second_parent_final =f"well versed in {formatted_second_parent} and I can help you with that!" if formatted_second_parent != "" else ""
cond_1 = f"I see that you are looking for {formatted_first} {formatted_second_parent_final}"

if pay_amount > 0:
    formatted_pay = f""" Hence, I am willing to offer you this service for {pay_amount*0.8} instead of {pay_amount}"""
else:
    formatted_pay = """In this instance and rather than providing you with a quote as I assume you are on a budget, I would prefer to ask you how much are you expecting to pay for this service? if it is reasonable, I would gladly offer you my services. """

if is_tutor:
    formatted_tutoring = """As I understood, you are seeking someone to teach you how to do things instead of providing you with the services. What I generally do for my clients would be that I would ask you to provide me details regarding a specific project you have in mind, and I would self-record myself doing it. This will enable you to skip the learning of basics and ancillary things and learn exactly what you want. You will have the video showing every movements and actions being made from scratch to the result. I will also provide you, if needed, the resulting work."""

if is_sample:
    formatted_sample = """Feel free to have a look at my samples:
    """

if is_payment_method:
    formatted_pay_method = f"""I am fine to accept payments with {pay_method[0]} as you have mentioned it within your posting.
    """
else:
    formatted_pay_method = f"""I accept payments through Paypal as it is the only platform that provides a buyer’s protection for you. 
I could send you an invoice for the service provided if needed. However, if you have other preferences, feel free to let me know.
    """

if is_contract_type:
    formatted_contract_type = f"""I am also willing, per your request to provide you/sign a {contract_type[0]} at your request.
    """

generated_email = f"""
{cl_email}
Greetings,
{cond_1}

My name is Karim and I am looking forward to fostering a long-term relationship with you or become your go-to service provider for any related future inquiries.
{formatted_pay}
{formatted_tutoring}
{formatted_sample}
{formatted_pay_method}
{formatted_contract_type}

If interested or have any further questions, feel free to reach me at your earliest convenience and by your preferred mean of communication.
Sincerely, 
Karim
karimafilal@hotmail.com
(408) 393-4260 """

#send_mail(subject, generated_email, "firstenaction@gmail.com", "CastirlaCorte56", "karimafilal@hotmail.com")
print(f"""Test
Title: {title}
body: {body}


generated email: {generated_email}
""")
#print(generated_email)

#all_info = run_api.append_non_exported("sent.csv", all_info).dropna().drop_duplicates()
#all_info.to_csv("sent.csv", index=False)

Test
Title: I need a graphic design
body: QR Code Link to This Post

I'm looking for a graphic designer who can alter some of my photos and add text
on them to look like a magazine cover or an add.
Someone who can write in different languages (Russian preferred) would be great.
Please send me your work to look at.
I will pay $15- $40 per picture, depends on how much text. I need about 6 pictures.



generated email: 

Greetings,
I see that you are looking for a graphic designer well versed in photoshop, indesign and russian and I can help you with that!

My name is Karim and I am looking forward to fostering a long-term relationship with you or become your go-to service provider for any related future inquiries.
In this instance and rather than providing you with a quote as I assume you are on a budget, I would prefer to ask you how much are you expecting to pay for this service? if it is reasonable, I would gladly offer you my services. 

Feel free to have a look at my samples:
    
I

In [25]:
parent_second_result

array(['photoshop', 'indesign', 'russian'], dtype=object)

In [26]:
import os

In [65]:
samples=[]
for file_name in parent_second_result:
    path = f"samples/{file_name}.txt"
    if os.path.isfile(path):
        with open(path,"r") as file:
            sample = file.read().strip()
        samples.append({file_name: sample})
        
        
string = """"""
for i in samples:
    string = string + (str(i)) + "\n"
string = (string.replace("{","").replace("}","").replace("'",""))
print(string)

photoshop: (before/after): https://1drv.ms/u/s!Ap7N4wuRlyn-gwMvwfXMiXdzXKP9?e=IMFbyB



In [10]:
all_info = run_api.append_non_exported("sent.csv", all_info).dropna().drop_duplicates()
all_info.to_csv("sent.csv", index=False)

In [ ]:
sent.csv file = 119 rows before clearing.

In [11]:
all_info

,title,body,compensation,pay_amount,amount,email
0,Part time Designer or Developer Needed,QR Code Link to This Post\n \n ...,compensation: Hourly pay,-1,hourly,acfb7d734a253ebb96c7b9f41edfb09d@gigs.craigsli...
1,Fitness influencer for Video Tutorials,QR Code Link to This Post\n \n ...,compensation: Videos for your social media,-1,your social media,0b33353750073c1880bce115475a4794@gigs.craigsli...
2,Writing facts about business,QR Code Link to This Post\n \n ...,compensation: candidate will be paid $15 for t...,15,this gig,af531808f7ab3b508658a46d945d2cb4@gigs.craigsli...
3,Windows 10 PC migrations,QR Code Link to This Post\n \n ...,compensation: Best hourly rate or per unit offer,-1,hourly,46ed13404c9136f6b64f4078d00761a4@gigs.craigsli...
4,Seeking models for photo project,QR Code Link to This Post\n \n ...,compensation: 1.5 hours + photos,-1,hour,b850f4e42f423db1bd169ad204a8a0b1@gigs.craigsli...
...,...,...,...,...,...,...
83,Fashion Week: Hiring Models,QR Code Link to This Post\n \n ...,compensation: $300-$500/a show,300,a show,e7b6342fc46b3b99bbd027524ff8885c@gigs.craigsli...
84,Custom Online Booking Website,QR Code Link to This Post\n \n ...,"compensation: $25,000/year",25000,year,26cd83a899d73dfd85c3a2d97ce2c038@gigs.craigsli...
85,Software Development Intern/StudentNeeded,QR Code Link to This Post\n \n ...,compensation: $15-$20/hr,15,hr,5df2dca2ab8935e98a7b466f91847f28@gigs.craigsli...
86,Experienced Marketing Writer Wanted,QR Code Link to This Post\n \n ...,"compensation: Hourly, commensurate with experi...",-1,hourly,a0c2332b57663d6c8f15dd07e6c22f82@gigs.craigsli...


In [9]:
all_info.to_csv("sent.csv", index=False)

In [16]:
all_info = run_api.append_non_exported("sent.csv", all_info).dropna().drop_duplicates()
all_info.to_csv("sent.csv", index=False)

In [24]:
#all_info = utils.compare_against_sent( all_info, pd.read_csv("sent.csv"), ["title", "body", "compensation"])
all_info.reset_index(drop=True)

,title,body,compensation,pay_amount,amount,email
0,Wanted: SEO Marketing Intern,QR Code Link to This Post\n \n ...,compensation: $13/Hr,13,hr,02f446323bc833f88aa0f373e01f51db@gigs.craigsli...
1,Financial Accounting,QR Code Link to This Post\n \n ...,compensation: 50/neg,50,neg,6bdc664aa7ac3ea2b1a72fab7ea42726@gigs.craigsli...
2,Seeking Photography Model,QR Code Link to This Post\n \n ...,compensation: Rate $10 to $18 per hour,10,hour,7a2b40a3efcc3a598761a61a733665f1@gigs.craigsli...
3,looking for some one to do my seo on website,QR Code Link to This Post\n \n ...,compensation: i am paying 50.00 for 5 hours of...,-1,hour,1a6b9335f3543aadaff8c3ff1379ef46@gigs.craigsli...
4,Old couple or family wanted to model T shirt,QR Code Link to This Post\n \n ...,compensation: $50 per person. $130 for a Family.,50,a Family.,b39cddefeb2630dc8870c44260b0bf27@gigs.craigsli...
5,Social media marketing,QR Code Link to This Post\n \n ...,compensation: $15 an hour very flexible schedule,15,hour,ndosh3@gmail.com
6,"Social Media Help in Chandler, AZ (Remote Work)",QR Code Link to This Post\n \n ...,compensation: Negotiable,-1,NA,43b5260524c5342bb093fb807dbee3a0@gigs.craigsli...
7,Graphic Designer for Poems + Photo Book,QR Code Link to This Post\n \n ...,compensation: to be discussed,-1,NA,c9cbdd1691a83ef18f496e45c223249d@gigs.craigsli...
8,Product testers needed for newly developed san...,QR Code Link to This Post\n \n ...,compensation: You will be reimbursed $24.99 - ...,24,NA,fddf7a78d5723726848eb5fe032fece3@gigs.craigsli...
9,Experienced Visual Effects Artist for a Short ...,QR Code Link to This Post\n \n ...,compensation: Pay rate is $600 flat.,600,NA,handsupmovie@gmail.com


In [103]:
columns = ["title","body","compensation"]
df2 = pd.read_csv("sent.csv")
df2

,title,body,compensation,pay_amount,amount,email
0,one,two,three,-1,s,s
1,Hi,Hello,fag,-1,s,s
2,smthing,ggg,editing,-1,s,v
3,Bob,ss,video,bobs,s,v
4,smthing,ggg,editing,bobs,s,v2


In [104]:
compare_against_sent(df1, df2, columns)

,title,body,compensation,pay_amount,amount,email
0,Bob1,ss,video,bobs,s,v


In [63]:
run_api.append_non_exported("sent.csv",df1).to_csv("sent.csv", index=False)

In [93]:
df1.drop([0,1])

,title,body,compensation,pay_amount,amount,email


In [25]:
import os

In [58]:
samples=[]
for file_name in parent_second_result:
    path = f"samples/{file_name}.txt"
    if os.path.isfile(path):
        try:
            with open(path,"r") as file:
                sample = file.read().strip()
            samples.append(sample + "\n\n")
        except:
            print("problem with file: ", file_name)

In [59]:
string = """"""
for i in samples:
    string += i
print(string)

Here are my samples for photoshop where you can see the before and after versions
 https://1drv.ms/u/s!Ap7N4wuRlyn-gwMvwfXMiXdzXKP9?e=IMFbyB

Feel free to have a look at my samples for wordpress: 

https://www.healthrevadvisors.com/
https://www.firstcallrealestatesolutions.com/
http://www.thinkcaicreators.com/
https://solidgroundwellness.com/�
https://laxfit.net/
https://www.heart2heartrelationships.com/




In [82]:
pd.read_csv("sent.csv")["body"][0]

'QR Code Link to This Post\n            \n        \nHello CL , \n\nWe are currently looking for Digital Marketers / Interns, LOCAL out of Phoenix for an IN HOUSE SEO/ Marketing role.\n\n This is Ideal if you are newer to the industry , a student , or having a hard time getting/ closing clients. \n\n\nI have launched Web Design and Marketing Agencies in the past, and they have done very well until they were sold. Moving to Phoenix, I was not expecting so many clients coming to ask me for work, so now I have decided to begin to build an Agency again. But the next step to build a reliable team that can work great together. \n\n\nCurrently we work from the home office (Co-Working Lounge) . Our team currently consists of 2 Graphic Designers out of LA , 2 In house web designers, and one Videographer in house.  Preferably we would like to work with someone who has some SEO experience , Social Media , and content/ creative writing skills. The ideal candidate must have an entrepreneurial drive 

In [19]:
x = pd.read_csv("sent.csv")["title"] + pd.read_csv("sent.csv")["body"]
for i in x: 
    if i in sent: 
        print(i)

In [69]:
pd.DataFrame({
    "title and body": ["looking for some one to do my seo on website", "3D Printing and Painting"]
})

,title and body
0,looking for some one to do my seo on website
1,3D Printing and Painting


In [58]:
pd.DataFrame(sent["gig"].append(pd.Series("MY NAME IS KASEM"), ignore_index=True)).to_csv("sent.csv", index=False)

In [113]:
sent = pd.read_csv("sent.csv")
sent

,gig
0,Wanted: SEO Marketing Intern\n\nQR Code Link t...
1,Financial Accounting\n\nQR Code Link to This P...
2,Seeking Photography Model\n\nQR Code Link to T...
3,looking for some one to do my seo on website\n...
4,Old couple or family wanted to model T shirt\n...
...,...
162,i need a graphic design\n\n\n\ni'm looking for...
163,i need a video urgently\n\n\n\ni'm this is a t...
164,NaN
165,NaN


In [2]:
import pandas as pd

In [9]:
Nfiles = pd.read_csv("sent_Nfile1.csv")
for text, link in zip(Nfiles["field1_Text_Text"], Nfiles["field1_Link_Link"]):
    Nfiles_sent = pd.DataFrame(Nfiles_sent['text&link'].append(pd.Series(text+link), ignore_index=True), columns = ["text&link"])

In [118]:
sent = pd.DataFrame(sent['gig'].append(pd.Series(raw_job_post.strip()), ignore_index=True), columns = ["gig"])
sent.to_csv("sent.csv", index=False)

In [123]:
"Fully Furnished Bed and Bath Available in Hayes Valley 2br/2ba Apt" in Nfiles.values

True

In [12]:
Nfiles_sent.to_csv("sent_Nfile1.csv", index = False)

,text&link
